In [86]:
import keras
from keras import layers

import numpy as np
import random
import io

In [87]:
path = "3heads + Town + Will.txt"#твори Тигролови, Місто
with io.open(path, encoding="utf-8") as f:
    text = f.read().lower()
text = text.replace("\n", " ")  
print("Corpus length:", len(text))

chars = sorted(list(set(text)))
print("Total chars:", len(chars))
char_indices = dict((c, i) for i, c in enumerate(chars))
indices_char = dict((i, c) for i, c in enumerate(chars))

maxlen = 40
step = 3
sentences = []
next_chars = []
for i in range(0, len(text) - maxlen, step):
    sentences.append(text[i : i + maxlen])
    next_chars.append(text[i + maxlen])
print("Number of sequences:", len(sentences))

x = np.zeros((len(sentences), maxlen, len(chars)), dtype="bool")
y = np.zeros((len(sentences), len(chars)), dtype="bool")
for i, sentence in enumerate(sentences):
    for t, char in enumerate(sentence):
        x[i, t, char_indices[char]] = 1
    y[i, char_indices[next_chars[i]]] = 1

Corpus length: 1201340
Total chars: 88
Number of sequences: 400434


In [88]:
model = keras.Sequential(
    [
        keras.Input(shape=(maxlen, len(chars))),
        layers.LSTM(128),
        layers.Dense(len(chars), activation="softmax"),
    ]
)
optimizer = keras.optimizers.RMSprop(learning_rate=0.01)
model.compile(loss="categorical_crossentropy", optimizer=optimizer)

In [89]:
def sample(preds, temperature=1.0):
    preds = np.asarray(preds).astype("float64")
    preds = np.log(preds) / temperature
    exp_preds = np.exp(preds)
    preds = exp_preds / np.sum(exp_preds)
    probas = np.random.multinomial(1, preds, 1)
    return np.argmax(probas)

In [90]:
epochs = 10
batch_size = 64
model.fit(x, y, batch_size=batch_size, epochs=epochs)


Epoch 1/10
6257/6257 ━━━━━━━━━━━━━━━━━━━━ 176s 28ms/step - loss: 2.3645
Epoch 2/10
6257/6257 ━━━━━━━━━━━━━━━━━━━━ 177s 28ms/step - loss: 1.9485
Epoch 3/10
6257/6257 ━━━━━━━━━━━━━━━━━━━━ 176s 28ms/step - loss: 1.8726
Epoch 4/10
6257/6257 ━━━━━━━━━━━━━━━━━━━━ 178s 28ms/step - loss: 1.8383
Epoch 5/10
6257/6257 ━━━━━━━━━━━━━━━━━━━━ 174s 28ms/step - loss: 1.8179
Epoch 6/10
6257/6257 ━━━━━━━━━━━━━━━━━━━━ 174s 28ms/step - loss: 1.7992
Epoch 7/10
6257/6257 ━━━━━━━━━━━━━━━━━━━━ 168s 27ms/step - loss: 1.7838
Epoch 8/10
6257/6257 ━━━━━━━━━━━━━━━━━━━━ 166s 27ms/step - loss: 1.7739
Epoch 9/10
6257/6257 ━━━━━━━━━━━━━━━━━━━━ 171s 27ms/step - loss: 1.7630
Epoch 10/10
6257/6257 ━━━━━━━━━━━━━━━━━━━━ 170s 27ms/step - loss: 1.7574


In [91]:
start_index = random.randint(0, len(text) - maxlen - 1)
for diversity in [0.2, 0.5, 1.0, 1.2]:
        print("...Diversity:", diversity)

        generated = ""
        sentence = text[start_index : start_index + maxlen]
        print('...Generating with seed: "' + sentence + '"')

        for i in range(400):
            x_pred = np.zeros((1, maxlen, len(chars)))
            for t, char in enumerate(sentence):
                x_pred[0, t, char_indices[char]] = 1.0
            preds = model.predict(x_pred, verbose=0)[0]
            next_index = sample(preds, diversity)
            next_char = indices_char[next_index]
            sentence = sentence[1:] + next_char
            generated += next_char

        print("...Generated: ", generated)
        print("-")

...Diversity: 0.2
...Generating with seed: "ященну мітлу, завжди нову й завжди бездо"


...Generated:  роту. а він почав поет поставали, що він подався, вирішив він поставав і почував старий сірко він подався в комусь від невеличка подалі. а він погане він подав поставити від старих і виродив на подобученням подивитись з погляданням на стовком подався. він подав подався. він загрично почував по свої подалі. подався, подався. а він подався він почував старий сірко, подався, що він за став і він пода
-
...Diversity: 0.5
...Generating with seed: "ященну мітлу, завжди нову й завжди бездо"
...Generated:  мену кримана і від свої загір. — покинув він сказав степан. — такої кожен він погадав про було свої пересторі часом на нім почував з мистецтвий стойні, — не подумав він просто випередний завмитає їхали. але його григорій поставали на старих старого силеною спокоють. — хлопець ане його не забула випитилася, а випадав до театрі, — поставали вона собачу й навіть з відчуттям на людей і коли все на роз
-
...Diversity: 1.0
...Generating with seed: "ященну мітлу, завжди нову й завжди

In [92]:
epochs = 10
batch_size = 64
model.fit(x, y, batch_size=batch_size, epochs=epochs)

Epoch 1/10
6257/6257 ━━━━━━━━━━━━━━━━━━━━ 166s 26ms/step - loss: 1.7554
Epoch 2/10
6257/6257 ━━━━━━━━━━━━━━━━━━━━ 165s 26ms/step - loss: 1.7517
Epoch 3/10
6257/6257 ━━━━━━━━━━━━━━━━━━━━ 161s 26ms/step - loss: 1.7504
Epoch 4/10
6257/6257 ━━━━━━━━━━━━━━━━━━━━ 161s 26ms/step - loss: 1.7497
Epoch 5/10
6257/6257 ━━━━━━━━━━━━━━━━━━━━ 161s 26ms/step - loss: 1.7511
Epoch 6/10
6257/6257 ━━━━━━━━━━━━━━━━━━━━ 161s 26ms/step - loss: 1.7607
Epoch 7/10
6257/6257 ━━━━━━━━━━━━━━━━━━━━ 161s 26ms/step - loss: 1.7542
Epoch 8/10
6257/6257 ━━━━━━━━━━━━━━━━━━━━ 160s 26ms/step - loss: 1.7629
Epoch 9/10
6257/6257 ━━━━━━━━━━━━━━━━━━━━ 158s 25ms/step - loss: 1.7574
Epoch 10/10
6257/6257 ━━━━━━━━━━━━━━━━━━━━ 157s 25ms/step - loss: 1.7679


In [93]:
start_index = random.randint(0, len(text) - maxlen - 1)
for diversity in [0.2, 0.5, 1.0, 1.2]:
        print("...Diversity:", diversity)

        generated = ""
        sentence = text[start_index : start_index + maxlen]
        print('...Generating with seed: "' + sentence + '"')

        for i in range(400):
            x_pred = np.zeros((1, maxlen, len(chars)))
            for t, char in enumerate(sentence):
                x_pred[0, t, char_indices[char]] = 1.0
            preds = model.predict(x_pred, verbose=0)[0]
            next_index = sample(preds, diversity)
            next_char = indices_char[next_index]
            sentence = sentence[1:] + next_char
            generated += next_char

        print("...Generated: ", generated)
        print("-")

...Diversity: 0.2
...Generating with seed: "ясте дерево. 	 	двадцать ще років тому, "
...Generated:  поки на насобок по настрій, завписав по себе почував він почував він від неї від них почував він почував по своїх почав по насів почував по всіх поступали від собі до неї та загалади, він почував він поки відповідав прикраси, що він почував він виписались на від наставання в собі старому від них до нас навіть на сторонках привітав на сторонки, що він почував по себе почував почувати по сторонку, щ
-
...Diversity: 0.5
...Generating with seed: "ясте дерево. 	 	двадцать ще років тому, "
...Generated:  як закричував себе пособали старих у стіні під них покинув, потім каже він замість перед усіма свого та захопилась на стояв погадала від тебе потім та почав та запитає в неї на виставка йому китайського така тільки перейманого виправали в виглядій щось при страхового викладувати, з стільки й мату до селомі і покинув у неї, як під собі з насловою собаки він про себе мане тільки землю стояли ві